This script is to add data general. 
Import all required tools

In [1]:
import ixmp
import message_ix
import numpy as np
import pandas as pd
import yaml

from collections.abc import Mapping
from itertools import repeat
from message_ix.models import MESSAGE_ITEMS
from message_ix.utils import make_df

%matplotlib inline

mp = ixmp.Platform('local')

<IPython.core.display.Javascript object>

This part is just to generate a dummy scenario as basis for adding technology parameters. Need to be removed once integrated

In [2]:
model = "Westeros Electrified"

base = message_ix.Scenario(mp, model=model, scenario="baseline")
scen = base.clone(
    model,
    "add_tech",
    "introducing the add_tech feature on MESSAGEix",
    keep_solution=False,
)
scen.check_out()

year_df = scen.vintage_and_active_years()
vintage_years, act_years = year_df["year_vtg"], year_df["year_act"]
model_horizon = scen.set("year")
country = "Westeros"


In [3]:
# Read DACCS technology data
with open('tech_data.yaml','r') as stream:
    tech_data = yaml.safe_load(stream)

# Set up dictionary of parameter indices list
par_idx = {}
data = {}

# Create dicitonary of parameter indices and data
for tech in set(tech_data):
    # add vintage and active years and update tech_data for each tech
    years_vtg_act = scen.vintage_and_active_years()
    years_vtg_act = years_vtg_act[years_vtg_act['year_vtg'] >= tech_data[tech]['year_init']]
    tech_data[tech]['year_vtg']= years_vtg_act['year_vtg'].to_list()
    tech_data[tech]['year_act']= years_vtg_act['year_act'].to_list()

    # collect parameter indices and update data
    par_idx.update({tech: {par: {idx: [] for idx in list(MESSAGE_ITEMS[par].get('idx_names'))} 
                           for par in set(tech_data[tech]) - set(['year_init','year_vtg','year_act'])
                          }})
    
    data.update({tech: {par: [] for par in list(par_idx[tech].keys())}})
    

In [6]:
# If those are not provided, then this block of code is needed to retrieve them from the data input
regions = []
emissions = []
times = []
modes = []
commodities = []
levels = []
relations = []

set_elements_dict = {
    'node_loc'   :{'data': regions,     'name':'node'},
    'emission'   :{'data': emissions,   'name':'emission'},
    'mode'       :{'data': modes,       'name':'mode'},
    'time'       :{'data': times,       'name':'time'},
    'commodity'  :{'data': commodities, 'name':'commodity'},
    'level'      :{'data': levels,      'name':'level'},
    'time_origin':{'data': times,       'name':'time'},
    'time_dest'  :{'data': times,       'name':'time'},
    'relation'   :{'data': relations,   'name':'relation'},
    'node_rel'   :{'data': regions,     'name':'node'},
}   

# investment cost


In [10]:
# DONOT DELETE
tec = 'DACCS'
par = 'emission_factor'

kwargs = {}
if all(idx in par_idx[tec][par] for idx in ['year_vtg','year_act']):
    kwargs = {
        'year_vtg': tech_data[tec]['year_vtg'],
        'year_act': tech_data[tec]['year_act']
    }
elif 'year_vtg' in par_idx[tech][par]:
    kwargs = {
        'year_vtg': sorted(set(tech_data[tec]['year_vtg']))
    }
else:
    kwargs = {
        'year_act': sorted(set(tech_data[tec]['year_act']))
    }
    # if 'year_rel' is present, the values are assumed from 'year_act' values
    if 'year_rel' in par_idx[tec][par]:
        kwargs.update(
            {'year_rel': sorted(set(tech_data[tec]['year_act']))}
        )

df = make_df(par,
             technology=tec,
             value=tech_data[tec][par]['value'],
             unit=tech_data[tec][par]['unit'],
             **kwargs)    

# create empty dataframe
idx_exp = [e for e in par_idx[tec][par] 
           if e not in ['technology','year_vtg','year_act',
                        'node_origin','node_dest',
                        'time_origin','time_dest',
                       ]]

for idx in idx_exp:
    default = (
        {e:1 for e in list(scen.set(set_elements_dict[idx]['name']))[1:]} 
        if idx in ['node_loc','mode']
        else {e:1 for e in list(scen.set(set_elements_dict[idx]['name']))}
    )
    
    listidx = list(tech_data[tec][par].get(idx,default))
    listdfidx = []
    for e in listidx:
        df1 = df.copy()
        df1[idx] = [e] * len(df)
        listdfidx.append(df1)
    df = pd.concat(listdfidx, ignore_index=True)

# assigning values for node and time related indices
for idx in df.columns:
    if idx in ['node_origin','node_dest']:
        df[idx] = df['node_loc']
    if idx in ['time_origin','time_dest']:
        df[idx] = df['time']

# Calculate value row-by-row multiplier
mult = []
for i in range(len(df)):
    # node_loc factor
    _node_loc = (tech_data[tec]
                  .get(par,{})
                  .get('node_loc',{})
                  .get(df.get('node_loc',{}).get(2),1)
                 )

    # year_vtg factor
    # _year_vtg = (1+rate)**delta_years
    _year_vtg = (((1+tech_data[tec]
                   .get(par,{})
                   .get('year_vtg',{})
                   .get('rate',0))
                  **(df['year_vtg'][i] - tech_data[tech]['year_init']))
                  if 'year_vtg' in df.columns else 1
                )

    # year_act factor
    # _year_act = (1+rate)**(year_act-year_vtg) if both years present
    # _year_act = (1+rate)**(year_act-first_active_year) if no year_vtg
    _year_act = (((1+tech_data[tec]
                    .get(par,{})
                    .get('year_act',{})
                    .get('rate',0))
                   **(df['year_act'][i]
                     -(df['year_vtg'][i] 
                       if 'year_vtg' in df.columns 
                       else tech_data[tech]['year_init'])))
                  if 'year_act' in df.columns else 1
                )

    # get mode multiplier from model_data
    _mode = (tech_data[tec]
             .get(par,{})
             .get('mode',{})
             .get(df.get('mode',{}).get(i),1)
            )

    mult.append(
            np.prod([_node_loc, _year_vtg, _year_act, _mode,])
        )

# index adjusted df
value = df['value'] * mult
value = [round(e,3) for e in value]
df['value'] = value

df

,node_loc,technology,year_vtg,year_act,mode,emission,value,unit
0,Westeros,DACCS,700,700,standard,CO2,1,tCO2/kWa
1,Westeros,DACCS,700,710,standard,CO2,1,tCO2/kWa
2,Westeros,DACCS,700,720,standard,CO2,1,tCO2/kWa
3,Westeros,DACCS,710,710,standard,CO2,1,tCO2/kWa
4,Westeros,DACCS,710,720,standard,CO2,1,tCO2/kWa
5,Westeros,DACCS,720,720,standard,CO2,1,tCO2/kWa
6,Easteros,DACCS,700,700,standard,CO2,1,tCO2/kWa
7,Easteros,DACCS,700,710,standard,CO2,1,tCO2/kWa
8,Easteros,DACCS,700,720,standard,CO2,1,tCO2/kWa
9,Easteros,DACCS,710,710,standard,CO2,1,tCO2/kWa


In [9]:
data

{'DACCS': {'level_cost_activity_soft_up': [],
  'input': [],
  'initial_new_capacity_up': [],
  'emission_factor': [],
  'growth_new_capacity_up': [],
  'construction_time': [],
  'technical_lifetime': [],
  'fix_cost': [],
  'capacity_factor': [],
  'growth_activity_lo': [],
  'level_cost_activity_soft_lo': [],
  'inv_cost': [],
  'output': [],
  'abs_cost_activity_soft_up': [],
  'growth_activity_up': [],
  'var_cost': []}}

# Below is old script

Creating basic dataframe to be filled in later

In [ ]:
data_expand['DACCS']['inv_cost'][0]

In [ ]:
all_params = []
for t in data_expand.keys():
    for par in data_expand[t].keys():
        if par not in all_params:
            all_params.append(par)

data_to_scenario = {par: [] for par in all_params}

for k,v in data_expand.items():
    for k2,v2 in v.items():
        data_to_scenario[k2].append(v2[0])

data_expand = {k: pd.concat(v) for k, v in data_to_scenario.items()}        

In [ ]:
data_expand['inv_cost']

In [ ]:
with pd.ExcelWriter('printed_data.xlsx', engine='xlsxwriter', mode='w') as writer:
    for sheet_name, sheet_data in data_expand.items():
        sheet_data.to_excel(writer, sheet_name=sheet_name, index=False)

In [ ]:
mp.close_db()